In [7]:
import numpy as np
import pandas as pd
import datetime

# Constants
c = 3e8  # Speed of light in m/s    
d = 0.5  # Probe diameter in mm
s = 6.2  # Feed strip length in mm
t = 1.8  # Thickness in mm

# Generate 100 random samples
frequencies = np.linspace(2e9, 9e9, 100)  # Frequency in Hz
dielectrics = np.linspace(2.1, 3.8, 100)  # Dielectric constant
heights = np.linspace(1.647, 2.19, 100)  # Substrate height in mm

# Lists to store results
results = []

for f, er, h in zip(frequencies, dielectrics, heights):
    # Wavelength in free space (converted to mm)
    lambda_0 = (c / f) * 1000  # Now in mm
    
    # Compute gap g (converted to mm correctly)
    g = (0.16 * lambda_0) - (h * np.sqrt(er))
    
    # Compute total height
    h_total = g + h  # Total height (air gap + substrate height)
    
    # Compute effective dielectric constant
    e_eff = (er + 1) / 2 + (er - 1) / 2 * (1 / np.sqrt(1 + 12 * (h_total / t)))
    
    # Compute W (Width of patch)
    W = (c / (2 * f)) * np.sqrt(2 / (e_eff + 1)) * 1000  # Converted to mm
    
    # Compute L_eff (Effective Length of patch)
    L_eff = (c / (2 * f * np.sqrt(e_eff))) * 1000  # Converted to mm
    
    # Compute fringing correction ΔL
    delta_L = 0.412 * h_total * ((e_eff + 0.3) * ((W / h_total) + 0.264)) / ((e_eff - 0.258) * ((W / h_total) + 0.8))
    
    # Compute L (Length of patch)
    L = L_eff - 2 * delta_L
    
    # Store results, now including d, s, and t
    results.append([f / 1e9, er, h, d, s, t, g, W, L])

# Convert to DataFrame
df = pd.DataFrame(results, columns=["Frequency (GHz)", "Dielectric", "Height (mm)", "Probe Diameter d (mm)", "Feed Strip Length s (mm)", "Thickness t (mm)", "Gap g (mm)", "Width W (mm)", "Length L (mm)"])

# Generate unique filename with timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"antenna_calculations_{timestamp}.xlsx"

# Save to Excel
df.to_excel(filename, index=False)

# Display first few rows
print("Results saved to:", filename)
print(df.head())


Results saved to: antenna_calculations_20250422_132226.xlsx
   Frequency (GHz)  Dielectric  Height (mm)  Probe Diameter d (mm)  \
0         2.000000    2.100000     1.647000                    0.5   
1         2.070707    2.117172     1.652485                    0.5   
2         2.141414    2.134343     1.657970                    0.5   
3         2.212121    2.151515     1.663455                    0.5   
4         2.282828    2.168687     1.668939                    0.5   

   Feed Strip Length s (mm)  Thickness t (mm)  Gap g (mm)  Width W (mm)  \
0                       6.2               1.8   21.613270     65.855058   
1                       6.2               1.8   20.776039     63.483012   
2                       6.2               1.8   19.992901     61.268142   
3                       6.2               1.8   19.258668     59.195369   
4                       6.2               1.8   18.568791     57.251484   

   Length L (mm)  
0      36.242995  
1      34.912171  
2      33.6

In [5]:
!pip install pandas numpy scikit-learn joblib



Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\program files\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib  # To save the model

# Load Data
df = pd.read_excel(filename)  # Load the generated file

# Define input (X) and output (y) variables
X = df[["Frequency (GHz)", "Dielectric", "Height (mm)", "Probe Diameter d (mm)", "Feed Strip Length s (mm)", "Thickness t (mm)"]]
y = df[["Gap g (mm)", "Width W (mm)", "Length L (mm)"]]

# Split into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Regressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate model performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)

# Save the trained model
joblib.dump(model, "antenna_model.pkl")
print("Model saved as antenna_model.pkl")

# Predict on new input
new_data = np.array([[5.5, 2.5, 1.8, 0.5, 6.2, 1.8]])  # Example input (Freq, Dielectric, Height, d, s, t)
predicted_values = model.predict(new_data)
print("Predicted Gap g, Width W, Length L:", predicted_values)


Mean Absolute Error: 0.22742417934782053
Mean Squared Error: 0.3327241712908571
Model saved as antenna_model.pkl
Predicted Gap g, Width W, Length L: [[ 8.56864362 29.87520314 16.29428047]]


C:\Users\reddy\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load Data
df = pd.read_excel(r"C:\Users\reddy\Desktop\Major Final Project\antenna_calculations_20250422_132226.xlsx")  # Load the generated Excel file

# Define input (X) and output (y) variables
X = df[["Frequency (GHz)", "Dielectric", "Height (mm)", "Probe Diameter d (mm)", "Feed Strip Length s (mm)", "Thickness t (mm)"]]
y = df[["Gap g (mm)", "Width W (mm)", "Length L (mm)"]]

# Split into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Regressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate model performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100  # Mean Absolute Percentage Error

# Display metrics
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R² Score: {r2:.4f}")


# Save the trained model
import joblib
joblib.dump(model, "antenna_model.pkl")
print("Model saved as antenna_model.pkl")

# Predict on new input
new_data = np.array([[5.5, 2.5, 1.8, 0.5, 6.2, 1.8]])  # Example input
predicted_values = model.predict(new_data)
print("Predicted Gap g, Width W, Length L:", predicted_values)


Mean Absolute Error (MAE): 0.2274
Mean Squared Error (MSE): 0.3327
R² Score: 0.9969
Model saved as antenna_model.pkl
Predicted Gap g, Width W, Length L: [[ 8.56864362 29.87520314 16.29428047]]


C:\Users\reddy\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load Data
df = pd.read_excel(r"C:\Users\reddy\Desktop\Major Final Project\antenna_calculations_20250306_203739.xlsx")  # Update filename if needed

# Define input (X) and output (y)
X = df[["Frequency (GHz)", "Dielectric", "Height (mm)", "Probe Diameter d (mm)", "Feed Strip Length s (mm)", "Thickness t (mm)"]]
y = df[["Gap g (mm)", "Width W (mm)", "Length L (mm)"]]

# Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize Data (Deep Learning works better with normalized data)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

# Build Neural Network Model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Input Layer
    keras.layers.Dense(64, activation='relu'),  # Hidden Layer
    keras.layers.Dense(32, activation='relu'),  # Hidden Layer
    keras.layers.Dense(y_train.shape[1])  # Output Layer (3 outputs: Gap g, Width W, Length L)
])

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train Model
history = model.fit(X_train, y_train, epochs=500, batch_size=10, validation_split=0.1, verbose=1)

# Evaluate Model
y_pred = model.predict(X_test)

# Convert predictions back to original scale
y_pred = scaler_y.inverse_transform(y_pred)
y_test_original = scaler_y.inverse_transform(y_test)

# Calculate Accuracy Metrics
mae = mean_absolute_error(y_test_original, y_pred)
mse = mean_squared_error(y_test_original, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_original, y_pred)
mape = np.mean(np.abs((y_test_original - y_pred) / y_test_original)) * 100

print(f"\nMean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

# Save Model
model.save("antenna_deep_learning_model.h5")
print("Model saved as antenna_deep_learning_model.h5")

# Predict New Data
new_data = np.array([[5.5, 2.5, 1.8, 0.5, 6.2, 1.8]])  # Example input
new_data_scaled = scaler_X.transform(new_data)
predicted_values = model.predict(new_data_scaled)
predicted_values_original = scaler_y.inverse_transform(predicted_values)

print("Predicted Gap g, Width W, Length L:", predicted_values_original)


C:\Users\reddy\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 1.0391 - mae: 0.7971 - val_loss: 0.4293 - val_mae: 0.5891
Epoch 2/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9119 - mae: 0.6477 - val_loss: 0.2742 - val_mae: 0.4349
Epoch 3/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.5258 - mae: 0.4714 - val_loss: 0.1614 - val_mae: 0.3272
Epoch 4/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3470 - mae: 0.4064 - val_loss: 0.0842 - val_mae: 0.2266
Epoch 5/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3561 - mae: 0.3879 - val_loss: 0.0445 - val_mae: 0.1697
Epoch 6/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2361 - mae: 0.3163 - val_loss: 0.0236 - val_mae: 0.1287
Epoch 7/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1222 - mae: 0.2575 - val_loss: 0.0152 - val_mae: 0.1062
Epoch 8/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1159 - mae: 0.2489 - val_loss: 0.0147 - val_mae: 0.0960
Epoch 9/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1058 - mae: 0.2509 


Mean Absolute Error (MAE): 0.3502
Mean Squared Error (MSE): 0.4137
Root Mean Squared Error (RMSE): 0.6432
R² Score: 0.9971
Model saved as antenna_deep_learning_model.h5


C:\Users\reddy\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
Predicted Gap g, Width W, Length L: [[ 7.1301293 24.57496   14.353025 ]]
